In [1]:
from itertools import compress
import os
import pandas as pd

import random
from sklearn.preprocessing import LabelEncoder

import torch
import torchaudio as ta
from torchcodec.decoders import AudioDecoder
import torchaudio.functional as AF
from torch.utils.data import Dataset, DataLoader

from functions.dl.data_classes import SpectroDataset, CombinedSpectroDataset, SpectroDataLoader

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [12]:
r_path ="./AudioTensors"
l_path = os.getcwd()

In [13]:
dir_files = os.listdir(l_path)
parquet_file = next(f for f in dir_files if f.endswith("_single.parquet"))
sound_df = pd.read_parquet(os.path.join(l_path, parquet_file))
sound_df = sound_df.drop(sound_df.loc[sound_df.label.isna()].index)

In [14]:
sound_df

,id,label,geometry
0,1000106,312,b'\x01\x01\x00\x00\x00\x8d\x02\xe1\xdcW\xd7NA\...
1,1000371,312,b'\x01\x01\x00\x00\x00\xf8=\xfb;\xbamPAkmD4quIA'
2,1001124,313,b'\x01\x01\x00\x00\x00\r?g\xbf\xfa\xc2OA\x1e\x...
3,1001127,313,b'\x01\x01\x00\x00\x00\xdeiL\x0c\n\xc3OA:\xfa\...
4,100114,231,b'\x01\x01\x00\x00\x00\xa3\x18|\xd0\xdbCPA\xb4...
...,...,...,...
157363,999439,322,b'\x01\x01\x00\x00\x00\xc8\xf7X\x98rKQA\xe4\xb...
157364,999547,312,b'\x01\x01\x00\x00\x00\xf9\xf1<\xae\xaemPA\xcc...
157365,99984,313,b'\x01\x01\x00\x00\x00\x16\xa1\xc9\x9f\xdf\xbf...
157366,99986,313,b'\x01\x01\x00\x00\x00\x16\xa1\xc9\x9f\xdf\xbf...


In [97]:
dawn_file = "F:/Persönliches/Git/BioOTon/points_single.parquet"
xeno_file = "F:/Persönliches/Git/BioOTon/xeno_points_single.parquet"
augmented_file = "F:/Persönliches/Git/BioOTon/augmented_points_single.parquet"

ds = CombinedSpectroDataset(dawn_file, xeno_file, augmented_file, device = device, denoised = True)

In [98]:
dawn_df = pd.read_parquet(dawn_file)
xeno_df = pd.read_parquet(xeno_file)
xeno_df.id = xeno_df.id.astype(int)
aug_df = pd.read_parquet(augmented_file)

In [99]:
i = 201000
ds.combined_file_paths[i], ds.combined_point_labels[i]

(WindowsPath('F:/Persönliches/Git/BioOTon/augmented_data_denoised/wn_979694_audio_dn.pt'),
 '211')

In [100]:
dawn_df.loc[dawn_df.id == int(ds.combined_file_paths[i].stem.replace("wn_","").replace("_audio_dn",""))]

,id,geometry,label


In [101]:
xeno_df.loc[xeno_df.id == int(ds.combined_file_paths[i].stem.replace("wn_","").replace("_audio_dn",""))]

,id,geometry,label
6856,979694,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...,211


In [102]:
aug_df.loc[aug_df.id == int(ds.combined_file_paths[i].stem.replace("wn_","").replace("_audio_dn",""))]

,id,label,geometry
15664,979694,211,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...
31391,979694,211,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...
47118,979694,211,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...
62845,979694,211,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...
78589,979694,211,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...
94316,979694,211,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...
110070,979694,211,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...
125797,979694,211,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...
141551,979694,211,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...
157305,979694,211,b'\x01\x01\x00\x00\x00\xf3\xab\xc0\x04\xae\xc9...
